In [1]:
import numpy as np
from tqdm import tqdm_notebook

from tools.data_loading import load_images, load_labels, dummy_code, load_images_resized
from tools.feature_learning import pins_generation
from tools.hog import hog
from tools.kernels import kernel_matrix
from tools.optimization import find_f
from tools.prediction import pred
from tools.process_images import process_images
from tools.quantization import kmeans
from tools.submission import labels_to_csv
from tools.visualization import imshow, dump_as_png, reshape_as_images

# Data loading

In [3]:
X_train = load_images_resized(type="train")
n_train = X_train.shape[0]
Y_labels_train = load_labels()
Y_train = dummy_code(Y_labels_train)
n_classes = Y_train.shape[1]
X_train = X_train - X_train.min(axis=0)
X_train = X_train / X_train.max(axis=0)

In [4]:
X_train2 = np.zeros((5000,63,63,3))
for i in range(3):
    X_train2[:,:,:,i] = X_train
image_list = X_train2

# Visual features

In [5]:
filter_sigma=0.1
filter_shape=5
hog_cell_size=8
disc_grid=16

In [6]:
n_images = image_list.shape[0]

In [7]:
hog_list = []
for i in tqdm_notebook(range(n_images), desc="Images"):
    image = image_list[i,:,:,:]
    hog_list.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid))
    
n_features = hog_list[0].size
X_hog = np.array(hog_list).reshape((n_images, n_features)) 


# Data separation

In [8]:
indices = np.random.permutation(X_hog.shape[0])
training_idx, test_idx = indices[:int(0.8*n_train)], indices[int(0.8*n_train):]

X_sample = X_hog[training_idx, :]
n_sample = X_sample.shape[0]
Y_sample = Y_train[training_idx,:]
Y_labels_sample = Y_labels_train[training_idx]

X_test = X_hog[test_idx, :]
n_test = X_test.shape[0]
Y_labels_test = Y_labels_train[test_idx]

# Training

In [9]:
kernel_type = "polynomial"
classifier_type = "svm"
degree = 6
constant = 0.1
lamb = 1

## Kernel choice

In [10]:
K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, degree=degree, constant=constant)

In [11]:
K_sample.shape

(4000, 4000)

## Classifier choice

In [12]:
alpha = np.zeros((n_classes, n_sample))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                           prob_type=classifier_type, lamb=lamb, n_iter=50000)

# Evaluation

### Training error

In [13]:
Y_pred_train = np.zeros((X_sample[:500].shape[0], n_classes))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    Y_pred_train[:, dig] = pred(X_sample, X_sample[:500], alpha[dig, :],
                          kernel_type=kernel_type, degree=degree, constant=constant)


Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
prec = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
print("The precision on the train set is of {}".format(prec))


The precision on the train set is of 1.0


### Testing error

In [ ]:
Y_pred = np.zeros((X_test.shape[0], n_classes))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    Y_pred[:, dig] = pred(X_sample, X_test, alpha[dig, :],
                          kernel_type=kernel_type, degree=degree, constant=constant)


Y_labels_pred = np.argmax(Y_pred, axis=1)
prec = np.mean(Y_labels_pred == Y_labels_test)
print("The precision on the test set is of {}".format(prec))


The precision on the test set is of 0.559


## Grid search from hell

### For polynomial

In [ ]:
degrees = [2, 3, 4, 5, 6]
n_degrees = len(degrees)
constants = [0.1, 1, 5, 50, 100]
n_constants = len(constants)
lambs = [0.01, 0.1, 1, 10, 100]
n_lambs = len(lambs)
train_prec = np.zeros((n_degrees, n_constants, n_lambs))
test_prec = np.zeros((n_degrees, n_constants, n_lambs))
for d in tqdm_notebook(range(n_degrees), desc="Degrees"):
    for c in tqdm_notebook(range(n_constants), desc="Constants"):
        # Kernel matrix computation
        K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, degree=degrees[d], constant=constants[c])
        for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
            # Gradient descent
            alpha = np.zeros((n_classes, n_sample))
            for dig in range(n_classes):
                alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                                       prob_type=classifier_type, lamb=lambs[l], n_iter=50000)
            # Precision on train set
            Y_pred_train = np.zeros((X_sample[:500].shape[0], n_classes))
            for dig in range(n_classes):
                Y_pred_train[:, dig] = pred(X_sample, X_sample[:500], alpha[dig, :],
                                      kernel_type=kernel_type, degree=degrees[d], constant=constants[c])


            Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
            train_prec[d, c, l] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
            # Precision on test set
            Y_pred = np.zeros((X_test.shape[0], n_classes))
            for dig in range(n_classes):
                Y_pred[:, dig] = pred(X_sample, X_test, alpha[dig, :],
                                      kernel_type=kernel_type, degree=degrees[d], constant=constants[c])


            Y_labels_pred = np.argmax(Y_pred, axis=1)
            test_prec[d, c, l] = np.mean(Y_labels_pred == Y_labels_test)

### For RBF

In [ ]:
sigmas = [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100, 500, 1000]
n_sigmas = len(sigmas)
lambs = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
n_lambs = len(lambs)
train_prec = np.zeros((n_sigmas, n_lambs))
test_prec = np.zeros((n_sigmas, n_lambs))
for s in tqdm_notebook(range(n_sigmas), desc="Sigmas"):
    # Kernel matrix computation
    K_sample = kernel_matrix(X_sample, kernel_type=kernel_type, sigma=sigmas[s])
    for l in tqdm_notebook(range(n_lambs), desc="Lambdas"):
        # Gradient descent
        alpha = np.zeros((n_classes, n_sample))
        for dig in range(n_classes):
            alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                                   prob_type=classifier_type, lamb=lambs[l], n_iter=50000)
        # Precision on train set
        Y_pred_train = np.zeros((X_sample[:500].shape[0], n_classes))
        for dig in range(n_classes):
            Y_pred_train[:, dig] = pred(X_sample, X_sample[:500], alpha[dig, :],
                                  kernel_type=kernel_type, sigma=sigmas[s])


        Y_labels_pred_train = np.argmax(Y_pred_train, axis=1)
        train_prec[s, l] = np.mean(Y_labels_pred_train == Y_labels_sample[:500])
        # Precision on test set
        Y_pred = np.zeros((X_test.shape[0], n_classes))
        for dig in range(n_classes):
            Y_pred[:, dig] = pred(X_sample, X_test, alpha[dig, :],
                                  kernel_type=kernel_type, sigma=sigmas[s])


        Y_labels_pred = np.argmax(Y_pred, axis=1)
        test_prec[s, l] = np.mean(Y_labels_pred == Y_labels_test)

In [ ]:
test_prec

In [ ]:
train_prec

# Prediction

In [ ]:
X_eval = load_images_resized(type="test")
X_eval = X_eval - X_eval.min(axis=0)
X_eval = X_eval / X_eval.max(axis=0)
n_eval = X_eval.shape[0]
X_eval2 = np.zeros((2000,63,63,3))
for i in range(3):
    X_eval2[:,:,:,i] = X_eval
image_list_eval = X_eval2

# Visual features for submission
hog_list_eval = []
for i in tqdm_notebook(range(n_eval), desc="Submission images"):
    image = image_list_eval[i,:,:,:]
    hog_list_eval.append(
        hog(
            image,
            filter_sigma=filter_sigma,
            filter_shape=filter_shape,
            hog_cell_size=hog_cell_size,
            disc_grid=disc_grid))
    
n_features = hog_list_eval[0].size
X_hog_eval = np.array(hog_list_eval).reshape((n_eval, n_features)) 

In [ ]:
Y_eval = np.zeros((n_eval, n_classes))
for dig in tqdm_notebook(range(n_classes), desc="Classes"):
    Y_eval[:, dig] = pred(X_sample, X_hog_eval, alpha[dig, :],
                          kernel_type=kernel_type, degree=degree, constant=constant)


Y_labels_eval = np.argmax(Y_eval, axis=1)


# Submission
labels_to_csv(Y_labels_eval, kernel=kernel_type, algo="svm_hog",user="Zac_big_images")